<a href="https://colab.research.google.com/github/ashu7103/LLM---Project-Generation/blob/main/LLM_Project_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Cell 1 – User Inputs
project_name = "ChatApp"  # e.g., input("Enter project name: ").strip()
project_type = "Java Spring Boot web app"  # e.g., input("Enter project type: ").strip()
project_description = "group chat option with new group chat creation option, websoket implemention, Create view too and good visuals"  # Optional


In [ ]:
# Cell 2 – Save Config to JSON
import json
import os

def define_project(project_name, project_type, project_description=""):
    config = {
        "project_name": project_name,
        "project_type": project_type,
        "description": project_description,
    }

    os.makedirs(project_name, exist_ok=True)
    config_path = os.path.join(project_name, "project_config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)

    print(f"✅ Project config saved at: {config_path}")
    return config

# Call the function using previously defined variables
config = define_project(project_name, project_type, project_description)


In [ ]:
# Cell 1 – System Prompt Template
structure_prompt_template = """
You are a senior software architect. Given the project name and type, generate a complete directory and file structure for this project.
Include necessary folders (e.g., src, main, controller, service, utils, resources), and file names such as controllers, services, models, config files, etc.
with there location where to arrage them
Constraints:
- The structure must match the standard conventions of the project type.
- Do not generate code, only file and folder names.
- Do not generate any comment or extra text

Project Name: {project_name}
Project Type: {project_type}
"""

# Build final prompt
structure_prompt = structure_prompt_template.format(
    project_name=project_name,
    project_type=project_type
)


In [ ]:
# Cell 2 – LLM call to generate structure using DeepSeek via OpenRouter

import requests

def generate_structure_from_prompt(prompt, api_key, model="deepseek/deepseek-r1-0528:free"):
    url = "https://openrouter.ai/api/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://yourapp.com",  # Optional
        "X-Title": "LLMProjectBuilder"          # Optional
    }

    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful coding assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']

# Usage:
# openrouter_api_key = "your-api-key"
# structure_output = generate_structure_from_prompt(structure_prompt, openrouter_api_key)


In [ ]:
deepseek_api_key = "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"
structure_output = generate_structure_from_prompt(structure_prompt, deepseek_api_key)

In [ ]:
def parse_structure_text(structure_text: str) -> list:
    import re
    lines = structure_text.strip().splitlines()
    path_stack = []
    result = []

    for line in lines:
        # Skip empty lines or non-path notes
        if not line.strip() or line.strip().startswith("###") or line.strip().startswith("Key Structure"):
            continue

        # Count tree level (number of │ or ├── or └──)
        level = line.count("│") + line.count("├") + line.count("└")

        # Clean line
        cleaned = re.sub(r"[│├└─]+", "", line).strip()

        # Adjust stack to current level
        if level < len(path_stack):
            path_stack = path_stack[:level]
        elif level > len(path_stack):
            # Fill missing levels with placeholder folders if needed
            while len(path_stack) < level:
                path_stack.append("unknown")

        path_stack = path_stack[:level]
        path_stack.append(cleaned)

        # Join path and store
        full_path = "/".join(path_stack)
        result.append(full_path)

    return result

# Usage
parsed_paths = parse_structure_text(structure_output)  # use the text from your previous DeepSeek response
for path in parsed_paths:
    print(path)


In [ ]:
import os

def extract_file_paths_from_structure(path_list: list) -> list:
    file_paths = []

    for line in path_list:
        line = line.strip()
        if line == "" or line.endswith("/"):
            continue  # skip directories
        if '.' in os.path.basename(line):
            normalized = os.path.normpath(line)
            file_paths.append(normalized)

    return file_paths


In [ ]:
import os

def create_files_from_paths(file_paths: list, base_dir: str = "."):
    for path in file_paths:
        full_path = os.path.join(base_dir, path)
        os.makedirs(os.path.dirname(full_path), exist_ok=True)
        with open(full_path, 'w') as f:
            f.write("")  # create empty file
    print(f"✅ Created {len(file_paths)} files under: {os.path.abspath(base_dir)}")


In [ ]:
name_of_directory_where_project= "chatAPP"

In [ ]:
len(parsed_paths)

In [ ]:

file_paths = extract_file_paths_from_structure(parsed_paths)
create_files_from_paths(file_paths, base_dir=name_of_directory_where_project)


In [ ]:
file_paths

In [ ]:
# !pip uninstall -y openai
!pip install openai==0.28
import openai
import os

In [ ]:
# !pip install --upgrade openai openrouter


In [ ]:

# 🔑 Replace this with your OpenRouter API key
openai.api_key = "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"
openai.api_base = "https://openrouter.ai/api/v1"
model = "deepseek/deepseek-r1-0528:free"  # You can also use mistralai/mixtral-8x7b, etc.


In [ ]:
def generate_code_with_openrouter(prompt, model="deepseek/deepseek-r1-0528:free"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a senior Java Spring Boot developer. Generate complete, clean, production-ready code."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ ERROR: {str(e)}"


In [ ]:
import re
for file_path in file_paths:
    print(f"\n✏️ Generating code for: {file_path}")

    prompt = f"""Generate full Java code for this file in a Spring Boot Book Rental Application:
            File: {file_path}
            Make sure it's compatible with other typical files in the same app.
            Do not include explanations — only code.
            Requirements:

            - Do not include unrelated explanations or markdown
            - Output only valid Java code for this file"""

    generated_code = generate_code_with_openrouter(prompt)

    if "```" in generated_code:
        # Extract only code between triple backticks
        code_blocks = re.findall(r"```(?:\w+)?\n(.*?)```", generated_code, re.DOTALL)
        generated_code = code_blocks[0].strip() if code_blocks else generated_code.strip()

    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    if "ERROR" not in generated_code:
        with open(file_path, "w") as f:
            f.write(generated_code)
        print(f"✅ Code written to: {file_path}")
    else:
        print(generated_code)


In [ ]:
import shutil

def zip_project_folder(folder_path, output_zip_path):
    shutil.make_archive(output_zip_path, 'zip', folder_path)
    print(f"📦 Project zipped at: {output_zip_path}.zip")


In [ ]:
# Zip the BookRentalApp directory
zip_project_folder("BookRentalApp", "BookRentalApp_output")
